In [1]:
import os 
os.chdir(r'F:\Racial_Block_Voting\dev_pip') 

In [2]:
from Racial_Block_Voting_tools import *
from vote_index import *
import pandas as pd

### Tests Of vote_index
#### Create Dataframe from CSVs
##### power_index(df, TotalPopCol, TargetPopCol, Party1, Party2, vIndex=None, filename=None)

In [7]:
df_Votes = pd.read_csv('F:\Racial_Block_Voting\BAF-al_2020.csv',dtype={'GEOID20_left':str,'COUNTYFP20':str})

''' Uncomment read_csv file to generate results 
     comapring current Alabama VRA district to the 2 
     proposed in Whole County Map '''

df_CD = pd.read_csv('F:\Racial_Block_Voting\AL_94_CD116_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('F:\Racial_Block_Voting\AL_94_WC_6.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('F:\Racial_Block_Voting\AL_94_WC_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
df_Votes = df_Votes[['VTDST20','COUNTYFP20','G20PRERTRU','G20PREDBID']]
df_Votes = df_Votes.drop_duplicates().reset_index(drop=True)
df_Votes = df_Votes.rename(columns={'VTDST20':'VTD','COUNTYFP20':'COUNTY'})

df_CD = df_CD[['VTD','COUNTY','P0010001','P0010004']]
df_CD = df_CD.groupby(['VTD','COUNTY']).sum()
df = df_CD.merge(df_Votes, how='inner',on=['VTD','COUNTY'])
df = df.drop_duplicates()
df = df.drop(columns=['COUNTY'])
df_sum = df.groupby(['VTD']).sum().reset_index()
df = df[['VTD']].merge(df_sum, on='VTD')
df = df.drop_duplicates()
assert len(df.index) != 0
assert len(df_Votes.index) != 0

#### vIndex = ssi (Shapley–Shubik)

In [4]:
df.head()

,VTD,P0010001,P0010004,G20PRERTRU,G20PREDBID
0,000001,6114,1628,2270,1467
3,000002,2711,1972,291,711
5,000003,4655,2367,859,243
8,000004,9617,6460,1067,2457
11,000005,2886,702,1077,461


In [8]:
power_index(df=df, TotalPopCol='P0010001', TargetPopCol='P0010004', Party1='G20PRERTRU', Party2='G20PREDBID', vIndex='ssi', filename='Test-ssi')
df_ssi = pd.read_csv('F:\Racial_Block_Voting\dev_pip\Test-ssi-Shapley–Shubik.csv')
df_ssi.head()

c:\Miniconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Cat,T_Count,T_Index
0,T90%,50,0.416667
1,T60%,50,0.416667
2,O90%,0,0.000000
3,O60%,10,0.083333
4,N50%,10,0.083333


#### vIndex = bpi (Banzhaf Power)

In [9]:
power_index(df=df, TotalPopCol='P0010001', TargetPopCol='P0010004', Party1='G20PRERTRU', Party2='G20PREDBID', vIndex='bpi', filename='Test-bpi')
df_bpi = pd.read_csv('F:\Racial_Block_Voting\dev_pip\Test-bpi-Banzhaf-Power.csv')
df_bpi.head()

c:\Miniconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Cat,T_Index,T_PowerShare,T_Critical_Vote
0,T90%,6,37.5,37.5
1,T60%,6,37.5,37.5
2,O90%,0,0.0,0.0
3,O60%,2,12.5,12.5
4,N50%,2,12.5,12.5
